In [34]:
import pandas as pd
import pandasql

from pandasql import sqldf
from IPython.display import display

import random

from faker import Faker

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
# Инициализация Faker
fake = Faker()

user_num = 100
product_num = 50
order_num = 500 
# Генерация датафрейма пользователей
users = []
for _ in range(user_num):
    user = {
        'user_id': fake.random_int(min=1, max=1000),
        'name': fake.name(),
        'email': fake.email(),
        'age': fake.random_int(min=18, max=65),
        'city': fake.city(),
        'country': fake.country()
    }
    users.append(user)
users_df = pd.DataFrame(users)

# Генерация датафрейма продуктов
products = []
for _ in range(product_num):
    product = {
        'product_id': fake.random_int(min=1, max=100),
        'name': fake.word(),
        'category': random.choice(['Electronics', 'Clothing', 'Home', 'Books']),
        'price': round(random.uniform(10, 1000), 2)
    }
    products.append(product)
products_df = pd.DataFrame(products)

# Генерация датафрейма заказов
orders = []
for _ in range(order_num):
    order = {
        'order_id': fake.random_int(min=1, max=10000),
        'user_id': random.choice(users_df['user_id']),
        'product_id': random.choice(products_df['product_id']),
        'quantity': random.randint(1, 10),
        'order_date': fake.date_between(start_date='-1y', end_date='today')
    }
    orders.append(order)
orders_df = pd.DataFrame(orders)


pysqldf = lambda q: sqldf(q, globals())

# tasks for sql

Общая выручка (Total Revenue): Рассчитайте общую выручку, суммируя стоимость каждого заказа, учитывая количество продуктов и их цены.

In [40]:
q = """
    SELECT sum(quantity * price) as total_profit
    FROM products_df as p 
    LEFT JOIN orders_df as o on o.product_id = p.product_id ;
    """

pysqldf(q)

,total_profit
0,1921467.46


Средний чек (Average Order Value): Рассчитайте среднюю стоимость заказа, разделив общую выручку на количество заказов.

In [50]:
q = """
    SELECT SUM(quantity * price) / SUM(price) AS mean_all_price, --среднюю стоимость товара h
           SUM(quantity * price) / COUNT(price) AS mean_prince -- среднюю цену товара
    FROM orders_df as o
    LEFT JOIN products_df as p ON p.product_id = o.product_id;
    """

pysqldf(q)

,mean_all_price,mean_prince
0,5.455415,2348.982225


Количество уникальных пользователей (Unique Users): Определите количество уникальных пользователей, основываясь на поле "user_id" в датафрейме пользователей.

In [56]:
q = """
    SELECT COUNT(DISTINCT user_id)
    FROM users_df
    """
pysqldf(q)

,COUNT(DISTINCT user_id)
0,95


Продукт с наибольшим количеством продаж (Best-selling Product): Определите продукт, который был продался наибольшее количество раз, исходя из количества продуктов в каждом заказе.

In [69]:
q = """
    SELECT sum(quantity) as most_sales_category, name 
    FROM orders_df as o 
    LEFT JOIN products_df as p ON o.product_id = p.product_id
    GROUP BY name
    ORDER BY most_sales_category DESC
    LIMIT 1
    """

pysqldf(q)

,most_sales_category,name
0,204,these


Средний возраст пользователей (Average User Age): Рассчитайте средний возраст пользователей на основе данных из датафрейма пользователей.

In [130]:
q = """
    SELECT  SUM(age) / COUNT(age) AS average_age
    FROM
    ( SELECT  SUM(age) / COUNT(age) as age
      FROM users_df 
      GROUP BY user_id ) AS age
    """

# q = """
#     SELECT AVG(u.age) AS average_age
#     FROM users_df as u
#     LEFT JOIN (SELECT DISTINCT user_id
#                FROM users_df) AS  d  

#           ON u.user_id = d.user_id   ;
#     """

pysqldf(q)

,average_age
0,41


Общее количество заказов по категориям продуктов (Total Orders by Product Category): Подсчитайте общее количество заказов для каждой категории продуктов.

In [133]:
q = """
    SELECT category,
           sum(quantity) as old_quantity
    FROM orders_df as o
    LEFT JOIN products_df as p ON p.product_id = o.product_id
    GROUP BY category
    """

pysqldf(q)

,category,old_quantity
0,Books,1224
1,Clothing,1162
2,Electronics,914
3,Home,1182


Топ-5 городов с наибольшим количеством пользователей (Top 5 Cities by User Count): Определите пять городов с наибольшим количеством пользователей на основе данных из датафрейма пользователей.

In [141]:
q = """
    SELECT country,
           COUNT(user_id) AS number_users
    FROM users_df
    GROUP BY country
    ORDER BY number_users DESC
    LIMIT 5
    """
pysqldf(q)

,country,number_users
0,Sierra Leone,3
1,Myanmar,3
2,Hungary,3
3,Vanuatu,2
4,Taiwan,2


Общее количество заказов за каждый месяц (Total Orders per Month): Разделите заказы по месяцам и рассчитайте общее количество заказов для каждого месяца.

In [151]:
q = """
    SELECT strftime('%m', order_date) AS month,
           COUNT(quantity) AS count
    FROM orders_df
    GROUP BY month ;
    """

pysqldf(q)

,month,count
0,01,35
1,02,46
2,03,37
3,04,41
4,05,37
5,06,45
6,07,53
7,08,41
8,09,33
9,10,43


Самый активный пользователь (Most Active User): Определите пользователя с наибольшим количеством заказов на основе данных из датафрейма заказов.

In [161]:
q = """
    SELECT u.user_id, 
           SUM(quantity) AS mau
    FROM users_df AS u
    LEFT JOIN orders_df as o
         ON u.user_id = u.user_id
    GROUP BY u.user_id
    ORDER BY mau DESC
    LIMIT 1
    """

pysqldf(q)

,user_id,mau
0,810,5590


Общая стоимость продаж по странам (Total Sales by Country): Рассчитайте общую стоимость продаж для каждой страны на основе данных из датафрейма пользователей и заказов.

In [157]:
q = """
    SELECT country, 
           SUM(price) as total_sales
    FROM users_df as u
    LEFT JOIN orders_df as o
         ON u.user_id = o.user_id
    LEFT JOIN products_df as p
         ON o.product_id = p.product_id
    GROUP BY country
    """

pysqldf(q)

,country,total_sales
0,Afghanistan,2267.34
1,Albania,2469.89
2,Angola,8296.25
3,Anguilla,6928.36
4,Antarctica (the territory South of 60 deg S),2420.04
...,...,...
75,United States of America,2539.61
76,Uzbekistan,969.59
77,Vanuatu,5835.93
78,Venezuela,7078.78
